In [143]:
import pandas
import requests
import io
import zipfile
import json

In [79]:
def get_zipped_csv_from_url(url : str, csv_name : str, num_rows : int) -> pandas.DataFrame:
    web_file = requests.get(url=url)
    zip = zipfile.ZipFile(io.BytesIO(web_file.content))
    #data = pandas.read_csv(zip.open(csv_name), nrows=num_rows,usecols=['Date received','Issue'])
    data = pandas.read_csv(zip.open(csv_name), nrows=num_rows)
    return pandas.DataFrame(data)

    #print(df)
df = get_zipped_csv_from_url("https://files.consumerfinance.gov/ccdb/complaints.csv.zip","complaints.csv",10)

In [80]:
id = "1228393702244134912"
endpoint = "https://api.twitter.com/2/tweets/"+id

def get_credentials(path: str) -> str:
    file = open(credential_path, "r")
    token = file.readline()
    bearer_token = "Bearer " + token
    return bearer_token


    
credential_path = "/Users/at/API Credentials/TwitterCreds.txt"
file = open(credential_path, "r")
#read first line for bearer
token = file.readline()
bearer_token = "Bearer " + token
headers = {"Authorization" : bearer_token}
response = requests.get(endpoint,headers=headers).json()
print(response)


{'data': {'id': '1228393702244134912', 'text': 'What did the developer write in their Valentine’s card?\n  \nwhile(true) {\n    I = Love(You);  \n}'}}


In [81]:
####-----DOES NOT WORK YET--- NEED upgraded access levels for Twitter API
search_url = "https://api.twitter.com/2/tweets/search/all"
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
credential_path = "/Users/at/API Credentials/TwitterCreds.txt"
creds = get_credentials(credential_path)
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {creds}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
response = connect_to_endpoint(search_url, query_params)
#print(response.json())

def twitter_request(str: credential_path, url: str, query_params :dict):
    creds = get_credentials(credential_path)




    

401


Exception: (401, '{\n  "title": "Unauthorized",\n  "type": "about:blank",\n  "status": 401,\n  "detail": "Unauthorized"\n}')

In [174]:
#--Twitter Auth and Request
# Twitter request Factory creates a factory that creates twitter api requests
# The factory is in charge of handling the credentials by retrieving the credentials (bearer token)
# from file and determining the url of the endpoint for the request.
# The actual request just requires a query
# Create a new factory for each request type
# I guess this is way to not have to retrieve credentials everytime you need to send a request

class twitter_request_factory:
       
    bearer_token : str
    url : str
    request_type : str
    def __init__(self, url: str, credential_path, request_type):
        self.url = url
        self.request_type = request_type
        self.bearer_token = get_credentials(credential_path)

    def get_credentials(path: str) -> str:
        file = open(credential_path, "r")
        token = file.readline()
        bearer_token = "Bearer " + token
        return bearer_token
    
    def bearer_oauth(self,r):
        """
        Method required by bearer token authentication.
        """
        token = bearer_oauth
        r.headers["Authorization"] = f"Bearer {token}"
        r.headers["User-Agent"] = self.request_type
        return r
            
    def twitter_request_all(self, query_params):
        response = requests.request("GET", self.url, auth=self.bearer_oauth, params=query_params)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()
   
    def twitter_request_get_recent_tweets(self, query_params):
        headers = {"Authorization" : self.bearer_token}
        response = requests.request("GET", self.url, params = query_params, headers=headers)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response
    
twit_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/all", "/Users/at/API Credentials/TwitterCreds.txt","v2FullArchiveSearchPython")

#print(twit_factory.twitter_request_all({'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}))

In [232]:
#----This block creates a recent_tweet_factory that is used to create recent tweet request based
# on one set of credentials that are stored in the factory
# Each time the loop iterates it produces a new request with the "next_token" from the previous request along with the expected payload

def tweet_reader():
    recent_tweet_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/recent", "/Users/at/API Credentials/TwitterCreds.txt","NONE")
    query_params = {'query':'cruise'}
    recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
    response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
    print(response)
    response_dict = json.loads(response)
    next_request_token = response_dict["meta"]
    next_request_token = next_request_token['next_token']
    i = 0
    while i < 10:
        new_query_params = {'query':'cruise', 'next_token':next_request_token}
        recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
        response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
        print(response)
        next_request_token = response_dict["meta"]
        next_request_token = next_request_token['next_token']
        tweet_data = response_dict["data"]
        
        i+=1




200
{
    "data": [
        {
            "id": "1480729463121846272",
            "text": "RT @ExhibitionistsK: Boys on a cruise #teamexhib  https://t.co/sCqKEdxika"
        },
        {
            "id": "1480729427503992832",
            "text": "RT @Flights2World: The Best Things to do in Munich Germany #travel #travelphotography #travelgram #travelblogger #flights #hotels #cruise #\u2026"
        },
        {
            "id": "1480729378749546496",
            "text": "@spharu_cruise \u304a\u306f\u3088\u3046\u2e1c(* \u0951\ua4b3 \u0951*  )\u2e1d\u22c6*"
        },
        {
            "id": "1480729350269976578",
            "text": "\u3010\u30c1\u30fc\u30e08\u3011\u300cSAPPORO SOUND CRUISE 2022\u300d\u306e\u51fa\u6f14\u30e1\u30f3\u30d0\u30fc\u6c7a\u5b9a\uff01\uff01 https://t.co/rjIfQFzy5j #AKB48"
        },
        {
            "id": "1480729340438622209",
            "text": "RT @YourNewHandler: 14 wanted to join in on the throat training fun. He will be on the Atlantis cruis

In [ ]:
import pymongo

client = pymango.MongoClient("mongodb://localhost:27017/")
database = client["MainDatabase"]
cruise_tweets = database["CruiseTweets"]

#----This block creates a recent_tweet_factory that is used to create recent tweet request based
# on one set of credentials that are stored in the factory
# Each time the loop iterates it produces a new request with the "next_token" from the previous request along with the expected payload

def tweet_reader():
    recent_tweet_factory = twitter_request_factory("https://api.twitter.com/2/tweets/search/recent", "/Users/at/API Credentials/TwitterCreds.txt","NONE")
    query_params = {'query':'cruise'}
    recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
    response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
    print(response)
    response_dict = json.loads(response)
    next_request_token = response_dict["meta"]
    next_request_token = next_request_token['next_token']
    i = 0
    while i < 10:
        new_query_params = {'query':'cruise', 'next_token':next_request_token}
        recent_tweet_request = recent_tweet_factory.twitter_request_get_recent_tweets(query_params)
        response = json.dumps(recent_tweet_request.json(),indent=4, sort_keys=True)
        print(response)
        next_request_token = response_dict["meta"]
        next_request_token = next_request_token['next_token']
        tweet_data = response_dict["data"]
        cruise_tweets.insert_many(tweet_data)
        i+=1




